In [1]:
import os
os.chdir('/home/rafalb/work/molecules/chemicalSmilesSpace/src')

In [2]:
import argparse
#from tabulate import tabulate
from pathlib import Path
import predictiveModel
import loadDecoder
import trainSmiles
import numpy as np
import random
from rdkit import Chem

Using TensorFlow backend.


In [3]:
predictiveModel = predictiveModel.unpicklePredictiveModel('./results/20191108/1/predictiveModel_CNN-LSTM_300000_LD256_0.00045_1024_512_256.pckl')

In [4]:
predictiveModel

{'model': 'staticDynmic_CNN-LSTM_300000_LD256_0.00045_1024_512_256.h5',
 'nCharSmiles': 100,
 'nCharSet': 27,
 'nStatic': 3,
 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'nLatent': 256,
 'char2indices': {' ': 0,
  '#': 1,
  '(': 2,
  ')': 3,
  '-': 4,
  '.': 5,
  '1': 6,
  '2': 7,
  '3': 8,
  '4': 9,
  '5': 10,
  '=': 11,
  'C': 12,
  'F': 13,
  'G': 14,
  'I': 15,
  'N': 16,
  'O': 17,
  'P': 18,
  'S': 19,
  'U': 20,
  'W': 21,
  '[': 22,
  ']': 23,
  'c': 24,
  'o': 25,
  's': 26},
 'indices2char': {0: ' ',
  1: '#',
  2: '(',
  3: ')',
  4: '-',
  5: '.',
  6: '1',
  7: '2',
  8: '3',
  9: '4',
  10: '5',
  11: '=',
  12: 'C',
  13: 'F',
  14: 'G',
  15: 'I',
  16: 'N',
  17: 'O',
  18: 'P',
  19: 'S',
  20: 'U',
  21: 'W',
  22: '[',
  23: ']',
  24: 'c',
  25: 'o',
  26: 's'}}

In [5]:
nCharInSmiles = predictiveModel['nCharSmiles']
nCharSet = predictiveModel['nCharSet']
nStatic = predictiveModel['nStatic']
weightsFile = predictiveModel['model']
char2indices = predictiveModel['char2indices']
indices2char = predictiveModel['indices2char']
nLatent = predictiveModel['nLatent']
print(weightsFile)

staticDynmic_CNN-LSTM_300000_LD256_0.00045_1024_512_256.h5


In [6]:
nLatent

256

In [7]:
convDefinition = {}
convDefinition['initialDimWidth'] = 7
convDefinition['initialDimDepth'] = 7
convDefinition['nCNNlayers'] = 6
convDefinition['expansionCoeff'] = 1.2
encoderDimensions = [convDefinition, nLatent]
decoderDimensions = [nLatent, 1024, 512, 256]


lr = 0.004

In [8]:
prePath = './results/20191108/1/'
encoder, decoder, autoencoder = loadDecoder.loadCNNdecoder(nCharInSmiles, nCharSet, nStatic, encoderDimensions, decoderDimensions, lr, prePath+weightsFile)

W0314 10:55:40.586178 140381342119744 deprecation_wrapper.py:119] From /home/rafalb/miniconda3/envs/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0314 10:55:40.600849 140381342119744 deprecation_wrapper.py:119] From /home/rafalb/miniconda3/envs/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0314 10:55:40.621382 140381342119744 deprecation_wrapper.py:119] From /home/rafalb/miniconda3/envs/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0314 10:55:40.662390 140381342119744 deprecation_wrapper.py:119] From /home/rafalb/miniconda3/envs/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.place

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputDynamic (InputLayer)       (None, 100, 27)      0                                            
__________________________________________________________________________________________________
conv_0 (Conv1D)                 (None, 94, 7)        1330        inputDynamic[0][0]               
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 88, 7)        350         conv_0[0][0]                     
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 88, 7)        28          conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Co

In [9]:
replacementDict = {'Br': 'G', 'Cl': 'U', '[nH]': 'W'}
replacementDictInv = {v: k for k, v in replacementDict.items()}

In [10]:
def createLatentVector(nLatent):
    latentVector = [random.random() for iii in range(nLatent)]
    latentVector = np.array(latentVector).reshape(-1, nLatent)
    return latentVector
    
def createPrediction(latentVector, nCharInSmiles, nCharSet, decoder):
    return decoder.predict(latentVector)[0].reshape(nCharInSmiles, nCharSet)

In [11]:
def createSmiles(prediction, nCharInSmiles, inidices2char):
    smiles = []
    for iii in range(nCharInSmiles):
        smiles.append(indices2char[np.argmax(prediction[iii, :])])
    return str.join('', smiles).strip()


In [12]:
def postprocesSmiles(smiles, replacementDict):
    for key in replacementDict.keys():
        smiles = smiles.replace(key, replacementDict[key])
    return smiles

In [13]:
def oneHotEncodeSmiles(smiles, char2indices, nCharSet):
    nLen = len(smiles)
    dynamicFeatures = np.zeros((1, nLen, nCharSet), dtype=np.float)
    print(dynamicFeatures.shape)

    for charidx, char in enumerate(smiles):
            dynamicFeatures[0, charidx, char2indices[char]] = 1
    return dynamicFeatures
    


In [ ]:
smilesOHE = oneHotEncodeSmiles('C1CCCCC1'.ljust(100), char2indices, nCharSet)
aePrediction = autoencoder.predict(smilesOHE)
enPrediction = encoder.predict(smilesOHE)


In [42]:
smilesOHE = oneHotEncodeSmiles('C1CCCCC1'.ljust(100), char2indices, nCharSet)
aePrediction = autoencoder.predict(smilesOHE)
enPrediction = encoder.predict(smilesOHE)


(1, 100, 27)


In [43]:
enPrediction

array([[-1.1063626 ,  0.8144736 ,  0.14468107, -3.515045  ,  0.7314535 ,
        -3.1514494 ,  3.5240848 ,  1.8090805 , -1.4440186 ,  0.03146052,
        -0.86594856,  0.84573895,  1.7286003 , -1.6170135 ,  3.1701772 ,
         0.03740982, -1.0524921 , -0.48057082,  0.9857323 ,  3.4204907 ,
         0.5226277 , -0.09111172, -2.0122318 , -5.0878725 ,  1.0248008 ,
        -1.8046011 , -1.6505071 ,  2.1828477 ,  0.5716951 ,  3.0284111 ,
        -1.6175008 ,  0.1873194 ,  2.232561  ,  1.0717583 , -0.24045539,
         3.9464974 , -0.8101385 ,  1.4966774 ,  2.205061  , -1.279932  ,
        -1.263951  , -0.28607768,  2.5346644 , -1.2767706 ,  0.15117174,
         2.7046118 , -3.8653808 , -0.5707974 , -0.25714505,  1.8080368 ,
        -0.8834489 ,  0.60084933, -0.38027442,  1.4088249 , -1.9101644 ,
        -1.4622592 ,  3.706072  , -5.2130556 , -2.213954  ,  0.19655098,
         1.1667867 , -0.68617857,  0.20594633, -0.04005382, -0.42738208,
         0.9174513 ,  1.7499793 , -1.9076576 , -0.7

In [45]:
def recoverSmiles(prediction, nCharSet, indices2char, replacementDictInv):
    smilesCode = createSmiles(prediction, nCharSet, indices2char)
    smilesCode = postprocesSmiles(smilesCode, replacementDictInv)
    return smilesCode

3

In [14]:
nSmiles = 100
correctSmiles = []
for iii in range(nSmiles):
    latentVector = createLatentVector(nLatent)
    prediction = createPrediction(latentVector, nCharInSmiles, nCharSet, decoder)
    smilesCode = createSmiles(prediction, nCharInSmiles, indices2char)
    smilesCode = postprocesSmiles(smilesCode, replacementDictInv)
    print(smilesCode)
    m = Chem.MolFromSmiles(smilesCode)
    if not(m is None):
        correctSmiles.append(smilesCode)
        print('!!!!!!!!!')



CC(c1cccClFc(F)c(F)c1Cl)C(NC(=O)O)c2-c1cc(Cl)[nH]c2ccc(CC(=O)NCCCC3)cc12
Cc1ccccc1c(())((F)cF2)C(1C(=O)C(F)(F)F)C11c1cccc(NS(=O)(O)CCC)cc1-
CCCC[nH]1c(N)c(N(F)F))c(C(C)NC(=O)c))Cc1ccc(F)cc1)[nH]2CCC(C==O)O)C))[nH]c1OC
CCc11c((S(F)(F))F)c1(C(C)C(=O)O)c1Cc1ccc(C)cc1-ccc(CCC(=O)NCC)[[nH][nH]]1
Cc1cc(C)c(O)c(Cl)c1cc1C)Cc1c(C)cc(C)c1)cc(C)ccccc1CCS(=O)(O)CO))ccc1
Cc1cccccc1)cc(C)c2c(C)[nH](F)c(=Oc[nH]2CCc1cccc1-c1cc(C)[nH]o1
Ccc1ccc2c1)cc(Nc1[nH]cc(CC(NS(=O)(=O)c1ccc[nH]1)c1cccc1C)[nH]2C
CC(c1cc(F)c(OO)c1)1c(CC(NC(=O)(F)F))ccc1(=O)[nH]1C)()CC2(=O)=O)CCccccc1
COCCc1cc[nH]c11c(Cl)c1ccc(OC)c(C(F)(F)F)c2c(F))c[nH]c[nH][nH]3)C2C
CCCC1c(=CN((FF)F)ccccOC)c(C=C)(F)(F)FFF)(F)cc1c1CSc(CC(=O)NC)))[[nH][nH]]1
CO(cccc(F)ccc1Cl)c1cc(C(C))C(F)(F)F)cccc(Cl)c(F)cc(CC==))C1C
Cc1ccccc11ccc(Cccccc(COc1C(=O)O))(c1ccc1C)[[nH]1cccc(C(=O)N2CC3)c[nH][nH]]1
C=C([nH]1c(F)c(Cl)c(Cl)c2c1OCNC((O)((FO)c1ccc(F)cc2[nH]1)CC1
Cc1c(Cl)ccc)ccOC)cc2c1CCN(C(=O)cc(C)[nH]cccc1)[nH]cccccc1CC(=O)NCCCCcccc1
CC(c1cc(F)c(N)cc1c1cc(C(F)

In [24]:
correctSmiles

['CCCCCc1c(NC2CCCCC2)Cc(CC)c(O)c2c(N)cccc12']

In [38]:
    correctSmiles

[]

In [15]:
m = Chem.MolFromSmiles(smiles)

NameError: name 'smiles' is not defined

In [25]:
m.GetNumAtoms()

True